**Importing libraries**

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import sklearn
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn.preprocessing import Imputer, StandardScaler, RobustScaler
from sklearn.feature_extraction import DictVectorizer
from functools import reduce
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_curve, auc
import itertools
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from statistics import stdev
from scipy.stats import ttest_rel

from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
#from mlxtend.plotting import plot_decision_regions
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

**Importing data**

From the database, we've selected columns describing the time (year, date, time of day) of discovery and containment, the size and size class of the fires, location (described by latitude and longitude, state) as well as the owner of the land where the fire originated.


Detailed column descriptions:

FIRE_YEAR = Calendar year in which the fire was discovered or confirmed to exist.

DISCOVERY_DATE = Date on which the fire was discovered or confirmed to exist.

DISCOVERY_DOY = Day of year on which the fire was discovered or confirmed to exist.

DISCOVERY_TIME = Time of day that the fire was discovered or confirmed to exist.

STAT_CAUSE_DESCR = Description of the (statistical) cause of the fire.

CONT_DATE = Date on which the fire was declared contained or otherwise controlled (mm/dd/yyyy where mm=month, dd=day, and yyyy=year).

CONT_DOY = Day of year on which the fire was declared contained or otherwise controlled.

CONT_TIME = Time of day that the fire was declared contained or otherwise controlled (hhmm where hh=hour, mm=minutes).

FIRE_SIZE = Estimate of acres within the final perimeter of the fire.

FIRE_SIZE_CLASS = Code for fire size based on the number of acres within the final fire perimeter expenditures (A=greater than 0 but less than or equal to 0.25 acres, B=0.26-9.9 acres, C=10.0-99.9 acres, D=100-299 acres, E=300 to 999 acres, F=1000 to 4999 acres, and G=5000+ acres).

LATITUDE = Latitude (NAD83) for point location of the fire (decimal degrees).

LONGITUDE = Longitude (NAD83) for point location of the fire (decimal degrees).

OWNER_DESCR = Name of primary owner or entity responsible for managing the land at the point of origin of the fire at the time of the incident.

STATE = Two-letter alphabetic code for the state in which the fire burned (or originated), based on the nominal designation in the fire report.

In [ ]:
connect = sqlite3.connect('FPA_FOD_20170508.sqlite')
data = pd.read_sql_query("SELECT FIRE_YEAR, DISCOVERY_DATE, DISCOVERY_DOY, DISCOVERY_TIME, STAT_CAUSE_DESCR, CONT_DATE, CONT_DOY, CONT_TIME, FIRE_SIZE, FIRE_SIZE_CLASS, LATITUDE, LONGITUDE, OWNER_DESCR, STATE FROM 'Fires'", connect)

Date conversion

In [ ]:
epoch = pd.to_datetime(0, unit='s').to_julian_date()
#Convert discovery date from julian to standard date
data['DISCOVERY_DATE'] = pd.to_datetime((data['DISCOVERY_DATE'] - epoch), unit='D', errors = 'ignore')
#Convert containment date from julian to standard date
data['CONT_DATE'] = pd.to_datetime((data['CONT_DATE'] - epoch), unit='D', errors = 'ignore') 

Take a look at what we've imported

In [ ]:
data.head()

In [ ]:
data['DISCOVERY_MONTH'] = pd.DatetimeIndex(data['DISCOVERY_DATE']).month
data['DISCOVERY_DAY_OF_WEEK'] = data['DISCOVERY_DATE'].dt.weekday_name
data['CONT_MONTH'] = pd.DatetimeIndex(data['CONT_DATE']).month
data['CONT_DAY_OF_WEEK'] = data['CONT_DATE'].dt.weekday_name

data['DISCOVERY_DATE_COMPLETE'] = pd.to_datetime(data.DISCOVERY_DATE.astype('str') + ' ' + data.DISCOVERY_TIME, errors='coerce')
data['CONT_DATE_COMPLETE'] = pd.to_datetime(data.CONT_DATE.astype('str') + ' ' + data.CONT_TIME, errors='coerce')

Collapsing the containment and discovery time differences to the single feature 'duration'

In [ ]:
data['DURATION_MINUTES'] = (data.CONT_DATE_COMPLETE - data.DISCOVERY_DATE_COMPLETE).astype('timedelta64[m]')

#Dropping unneeded columns
data.drop(['DISCOVERY_DATE', 'CONT_DATE', 'CONT_TIME', 'CONT_DOY', 'CONT_MONTH', 'CONT_DAY_OF_WEEK'], axis=1, inplace=True)
data.drop(['DISCOVERY_DATE_COMPLETE', 'CONT_DATE_COMPLETE'], axis=1, inplace=True)
data.head()

In [ ]:
print(data.dtypes)

In [ ]:
data.FIRE_YEAR = data.FIRE_YEAR.astype(float)
data.DISCOVERY_DOY = data.DISCOVERY_DOY.astype(float)
data.DISCOVERY_TIME = data.DISCOVERY_TIME.astype(float)

#data.FIRE_SIZE = data.FIRE_SIZE.astype(float)
data.FIRE_SIZE_CLASS = data.FIRE_SIZE_CLASS.astype('category')
#data.LATITUDE = data.LATITUDE.astype(float)
#data.LONGITUDE = data.LONGITUDE.astype(float)
data.OWNER_DESCR = data.OWNER_DESCR.astype('category')
data.STATE = data.STATE.astype('category')
data.DISCOVERY_MONTH = data.DISCOVERY_MONTH.astype(float)

def convert_days_int(df_column):
    day_numbers=[]
    for day in df_column:  #converting categorical ordinal Strings correctly to ints
        if day=='Monday':#values 0-6, so Monday=0
            day_numbers.append('0')
        elif day=='Tuesday':
            day_numbers.append('1')
        elif day=='Wednesday':
            day_numbers.append('2')
        elif day=='Thursday':
            day_numbers.append('3')
        elif day=='Friday':
            day_numbers.append('4')
        elif day=='Saturday':
            day_numbers.append('5')
        else:  #previously we have seen that there are none missing, so no need for another case
            day_numbers.append('6')
    return day_numbers

data.DISCOVERY_DAY_OF_WEEK = convert_days_int(data.DISCOVERY_DAY_OF_WEEK)
data.DISCOVERY_DAY_OF_WEEK = data.DISCOVERY_DAY_OF_WEEK.astype(float)

cause_cat=[]
for row in data.STAT_CAUSE_DESCR:
    if row in ['Debris Burning', 'Arson', 'Campfire', 'Children', 'Smoking']:
        cause_cat.append('Human-direct')
    else:
        cause_cat.append('Natural/Human-indirect')

data['CAUSE_CATEGORY'] = cause_cat
data.CAUSE_CATEGORY = data.CAUSE_CATEGORY.astype('category')
data.drop(['STAT_CAUSE_DESCR'], axis=1, inplace=True)

In [ ]:
data.head()

For more than half of the fires, the owner of the land is unspecified, so it might be hard to infer this information - need to explore more. One-hot coding will likely help here.

In [ ]:
data.describe(include='all')

Shuffling and splitting the dataset

In [ ]:
features=data.drop(['CAUSE_CATEGORY'], axis=1)
targets=data['CAUSE_CATEGORY']
#with shuffling, seed, for replicability
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.4, random_state=0)

In [ ]:
print(X_train.isnull().sum())
print('train set size \n',X_train.shape)
print('\n')
#print(X_test.isnull().sum())
#print('test set size \n',X_test.shape)

In [ ]:
#Majority class percentage
y_train.values.describe()

In [ ]:
#Handling missing values - only after splitting
#Might want to test dropping instances with missing values from train set - need to explore effect on result
#Fill missing values with median values instead of deleting instances
#Might be a good idea to look at distributions

#Preprocessing pipeline
#For trees, forests only missing value handling and categorical conversion to one-hot-coding applies

class ColumnExtractor(TransformerMixin):

    def __init__(self, cols):
        self.cols = cols

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xcols = X[self.cols]
        return Xcols

class DFFeatureUnion(TransformerMixin):
    # FeatureUnion but for pandas DataFrames

    def __init__(self, transformer_list):
        self.transformer_list = transformer_list

    def fit(self, X, y=None):
        for (name, t) in self.transformer_list:
            t.fit(X, y)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xts = [t.transform(X) for _, t in self.transformer_list]
        Xunion = reduce(lambda X1, X2: pd.merge(X1, X2, left_index=True, right_index=True), Xts)
        return Xunion


class DFImputer(TransformerMixin):
    # Imputer but for pandas DataFrames

    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.imp = None
        self.statistics_ = None

    def fit(self, X, y=None):
        self.imp = Imputer(strategy=self.strategy)
        self.imp.fit(X)
        self.statistics_ = pd.Series(self.imp.statistics_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Ximp = self.imp.transform(X)
        Xfilled = pd.DataFrame(Ximp, index=X.index, columns=X.columns)
        return Xfilled


class DFStandardScaler(TransformerMixin):
    # StandardScaler but for pandas DataFrames

    def __init__(self):
        self.ss = None
        self.mean_ = None
        self.scale_ = None

    def fit(self, X, y=None):
        self.ss = StandardScaler()
        self.ss.fit(X)
        self.mean_ = pd.Series(self.ss.mean_, index=X.columns)
        self.scale_ = pd.Series(self.ss.scale_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xss = self.ss.transform(X)
        Xscaled = pd.DataFrame(Xss, index=X.index, columns=X.columns)
        return Xscaled
    
class DFRobustScaler(TransformerMixin):
    # RobustScaler but for pandas DataFrames

    def __init__(self):
        self.rs = None
        self.center_ = None
        self.scale_ = None

    def fit(self, X, y=None):
        self.rs = RobustScaler()
        self.rs.fit(X)
        self.center_ = pd.Series(self.rs.center_, index=X.columns)
        self.scale_ = pd.Series(self.rs.scale_, index=X.columns)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xrs = self.rs.transform(X)
        Xscaled = pd.DataFrame(Xrs, index=X.index, columns=X.columns)
        return Xscaled
    
class Log1pTransformer(TransformerMixin):

    def fit(self, X, y=None):
        # stateless transformer
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xlog = np.log1p(X)
        return Xlog
    
class DummyTransformer(TransformerMixin):

    def __init__(self):
        self.dv = None

    def fit(self, X, y=None):
        # assumes all columns of X are strings
        Xdict = X.to_dict('records')
        self.dv = DictVectorizer(sparse=False)
        self.dv.fit(Xdict)
        return self

    def transform(self, X):
        # assumes X is a DataFrame
        Xdict = X.to_dict('records')
        Xt = self.dv.transform(Xdict)
        cols = self.dv.get_feature_names()
        Xdum = pd.DataFrame(Xt, index=X.index, columns=cols)
        # drop column indicating NaNs
        nan_cols = [c for c in cols if '=' not in c]
        Xdum = Xdum.drop(nan_cols, axis=1)
        return Xdum

cat_features=['FIRE_SIZE_CLASS','OWNER_DESCR','STATE']
num_features=['FIRE_YEAR','DISCOVERY_DOY','DISCOVERY_TIME','LATITUDE','LONGITUDE','DISCOVERY_MONTH','DISCOVERY_DAY_OF_WEEK']
num_features_log=['DURATION_MINUTES', 'FIRE_SIZE']

#pipeline = make_pipeline(DFFeatureUnion([(make_pipeline(ColumnExtractor(cat_features),DummyTransformer())), (make_pipeline(ColumnExtractor(num_features),DFImputer(strategy='median')))]))
pipeline = Pipeline([
    ('features', DFFeatureUnion([
        ('categoricals', Pipeline([
            ('extract', ColumnExtractor(cat_features)),
            ('dummy', DummyTransformer())
        ])),
        ('numerics', Pipeline([
            ('extract', ColumnExtractor(num_features)),
            ('imputer', DFImputer(strategy='median'))
        ])),
        ('numerics', Pipeline([
            ('extract', ColumnExtractor(num_features_log)),
            ('imputer', DFImputer(strategy='median')),
            ('log', Log1pTransformer())
        ]))
    ])),
    ('scale', DFStandardScaler())
])



In [ ]:
x_proc=pipeline.fit_transform(X_train)
#x_proc.head()
y_train.head()

In [ ]:
#x_proc.describe(include='all')

In [ ]:
#print(x_proc.isnull().sum())
#print('\n')
#print(test.isnull().sum())

**Exploration**

In [ ]:
#data.drop(['DURATION_MINUTES','FIRE_SIZE'], axis=1).plot(kind='box', figsize=(15,15))
cat_features=['FIRE_SIZE_CLASS','OWNER_DESCR','STATE']
num_features=['FIRE_YEAR','DISCOVERY_DOY','DISCOVERY_TIME','LATITUDE','LONGITUDE','DISCOVERY_MONTH','DISCOVERY_DAY_OF_WEEK','DURATION_MINUTES', 'FIRE_SIZE']

x_proc[num_features].plot(kind='box', figsize=(15,15))

A lot of outliers with duration, fire size - try log transform, maybe cap duration, as the furthest ones appear to be errors

In [ ]:
#data['CAUSE_CATEGORY'].value_counts().plot(kind='bar',color='red')
#plt.show()
#Noticable imbalance, but better than before

In [ ]:
#data['DISCOVERY_DAY_OF_WEEK'].value_counts().plot(kind='bar',color='red')
#plt.show()

In [ ]:
#data['STATE'].value_counts().head(n=10).plot(kind='bar',color='red')
#plt.show()

**Correlation matrix**

Only for numerical values, doesn't make much sense to correlate non-ordinal categorical features (e.g. state)

In [ ]:
#def plot_corr(df,size=15):
#    corr = df.corr()  #the default method is pearson
#    fig, ax = plt.subplots(figsize=(size, size))
#    ax.matshow(corr,cmap=plt.cm.Reds)
#    plt.xticks(range(len(corr.columns)), corr.columns)
#    plt.yticks(range(len(corr.columns)), corr.columns)
#    for tick in ax.get_xticklabels():
#        tick.set_rotation(45)    
#    plt.show()

#plot_corr(x_proc)

**Here's how to get data ready for fitting models:**

In [ ]:
trainx=pipeline.fit_transform(X_train)
trainy=y_train.cat.codes

testx=pipeline.transform(X_test)
testy=y_test.cat.codes

#From here it should be straighforward to fit initial models for testing

In [ ]:
#Recursive feature elimination
# Create the RFE object and compute a cross-validated score.
fe_tree = DecisionTreeClassifier()
# The "accuracy" scoring is proportional to the number of correct classifications
rfecv = RFECV(estimator=fe_tree, step=1, scoring='accuracy', verbose=1, n_jobs=-1, cv=10)
rfecv.fit(trainx[:10000], trainy[:10000])

In [ ]:
print("Optimal number of features : %d" % rfecv.n_features_)

# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (no of correct classifications)")
plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_, color='red')
plt.show()
#Since there is no significant increase in performance with using more than ~40 features, 
#take those top 40, which may facilitate simpler model, which may generalize better

In [ ]:
#Gridsearch for hyperparameters
tree_pipeline=make_pipeline(DecisionTreeClassifier())
tree_pipeline.named_steps.keys()
tree_params=dict(decisiontreeclassifier__criterion=['entropy','gini'], decisiontreeclassifier__max_depth=[15,17,20,22,25])
tree_grid_search=GridSearchCV(estimator=tree_pipeline, param_grid=tree_params, verbose=1, n_jobs=-1, cv=5)
tree_grid_search.fit(trainx, trainy)

In [ ]:
tree_grid_search.best_params_

In [ ]:
forrest_pipeline=make_pipeline(RandomForestClassifier())
forrest_pipeline.named_steps.keys()
forrest_params=dict(randomforestclassifier__n_estimators=[30,35], randomforestclassifier__max_depth=[25,30,35])
forrest_grid_search=GridSearchCV(estimator=forrest_pipeline, param_grid=forrest_params, verbose=1, n_jobs=-1, cv=5)
forrest_grid_search.fit(trainx, trainy)

In [ ]:
forrest_grid_search.best_params_

**Decision Tree Classifier**

In [ ]:
tree = DecisionTreeClassifier(max_depth=17)
tree.fit(trainx, trainy)
tree_test_predicted = tree.predict(testx)

print('Decision tree accuracy: ', accuracy_score(testy, tree_test_predicted))

tree_train_predicted = tree.predict(trainx)
print('Decision tree train accuracy: ', accuracy_score(trainy, tree_train_predicted))

In [ ]:
#y_predicted=tree.predict(testx)

In [ ]:
#testy.values
#y=testy.ravel()
#testy.ravel()[1]
#y_predicted[1]
#y_predicted= tree.predict_proba(testx)
#y_predicted[:,1]
#testy.values
#testy[:10].values
#y_predicted[:10,1]
#targets.shape[0]
#testy[:10].values
#testy.values

In [ ]:
tree_predicted= tree.predict_proba(testx)
n_classes=3
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(testy.values, tree_predicted[:,1])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='black', lw=lw/2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

**Random Forest Classifier**

In [ ]:
forrest = RandomForestClassifier(max_depth=30, n_estimators=70, max_features=40)
forrest.fit(trainx, trainy)
forrest_test_predicted = forrest.predict(testx)

print('Random forest accuracy: ', accuracy_score(testy, forrest_test_predicted))

forrest_train_predicted = forrest.predict(trainx)
print('Random forest train accuracy: ', accuracy_score(trainy, forrest_train_predicted))

In [ ]:
forrest_predicted= forrest.predict_proba(testx)
n_classes=3
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(testy.values, forrest_predicted[:,1])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='red',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='black', lw=lw/2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

**Define confusion matrix function**

In [ ]:
forrest_predictions=forrest.predict(testx)

In [ ]:
tree_predictions=tree.predict(testx)

In [ ]:
class_names=['Direct','Natural/ Indirect']
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Reds):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Compute confusion matrix
forrest_cnf_matrix = confusion_matrix(testy.values, forrest_predictions)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(forrest_cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()



# Compute confusion matrix
tree_cnf_matrix = confusion_matrix(testy.values, tree_predictions)
np.set_printoptions(precision=2)

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(tree_cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

In [ ]:
basic_tree = DecisionTreeClassifier()
basic_tree.fit(trainx, trainy)
basic_tree_test_predicted = basic_tree.predict(testx)

print('Decision tree accuracy: ', accuracy_score(testy, basic_tree_test_predicted))

basic_tree_train_predicted = basic_tree.predict(trainx)
print('Decision tree train accuracy: ', accuracy_score(trainy,basic_tree_train_predicted))

In [ ]:
basic_tree_scores = cross_val_score(basic_tree, trainx, trainy, cv=5, verbose=1, n_jobs=-1)

tree_scores = cross_val_score(tree, trainx, trainy, cv=5, verbose=1, n_jobs=-1)

forrest_scores = cross_val_score(forrest, trainx, trainy, cv=5, verbose=1, n_jobs=-1)

In [ ]:
print('Cross validation accuracy scores:')
print('Tree with default hyperparameters')
print(basic_tree_scores.tolist())
print('Tree with tuned hyperparameters')
print(tree_scores.tolist())
print('Random forest with tuned hyperparameters')
print(forrest_scores.tolist())
print('')
print("95% confidence intervals:")
print("Basic tree accuracy: %0.5f (+/- %2f), stdev=%f" % (basic_tree_scores.mean(), basic_tree_scores.std() * 2, stdev(basic_tree_scores)))
print("Tree accuracy: %0.5f (+/- %2f), stdev=%f" % (tree_scores.mean(), tree_scores.std() * 2, stdev(tree_scores)))
print("Forrest accuracy: %0.5f (+/- %2f), stdev=%f" % (forrest_scores.mean(), forrest_scores.std() * 2, stdev(forrest_scores)))

In [ ]:
#Claims:
#1 (tuned) tree accuracy mean>basic_tree accuracy mean
#2 Forrest accuracy mean>tree accuracy mean
# take alpha the significance level=0.01

print(ttest_rel(tree_scores,basic_tree_scores).pvalue/2)
print(ttest_rel(forrest_scores,tree_scores).pvalue/2)

print(ttest_rel(tree_scores,basic_tree_scores))
print(ttest_rel(forrest_scores,tree_scores))
print('Since t>0 and p/2 < alpha, in both cases the null hypotheses are rejected (the null hypotheses are that mean value of differences is 0), so we accept the alternative hypotheses, which are the original claims')

In [ ]:
#Accuracy bars with confidence error bars
N = 3
means = (basic_tree_scores.mean(), tree_scores.mean(), forrest_scores.mean())
errs = (basic_tree_scores.std()*2, tree_scores.std()*2, forrest_scores.std()*2)
ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, means, width, color='r', yerr=errs)
ax.set_ylabel('Accuracy')
ax.set_title('Scores by models with 95% confidence interval error bars')
ax.set_xticks(ind )
ax.set_xticklabels(('Tree default', 'Tree tuned hyp.', 'Random forrest'))
ax.set_ylim(0.724,0.802)

plt.show()

In [ ]:
#Accuracy bars with confidence error bars
N = 3
means = (basic_tree_scores.mean(), tree_scores.mean(), forrest_scores.mean())
errs = (basic_tree_scores.std(), tree_scores.std(), forrest_scores.std())
ind = np.arange(N)  # the x locations for the groups
width = 0.35       # the width of the bars
fig, ax = plt.subplots()
rects1 = ax.bar(ind, means, width, color='r', yerr=errs)
ax.set_ylabel('Accuracy')
ax.set_title('Scores by models with standard deviation error bars')
ax.set_xticks(ind )
ax.set_xticklabels(('Tree default', 'Tree tuned hyp.', 'Random forrest'))
ax.set_ylim(0.724,0.802)

plt.show()